#### Round 2 chemical selection

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os
import sys
from scipy.spatial.distance import pdist, squareform

In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
processed_dir = TOP +'data/processed/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

In [20]:
LIB = TOP+'src/models/'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

In [21]:
from model_functions import *

In [8]:
df = pd.read_csv(interim_dir+'tsca_medoid_approach_round2_2feb2023.csv')

In [9]:
df.shape

(14247, 142)

In [10]:
df = df.iloc[:,2:]

In [14]:
df.final_picks.count()

318

In [18]:
[e for e in df.columns if 'smi' in e]

['smiles', 'qsar_ready_smiles']

In [19]:
df[['smiles', 'qsar_ready_smiles']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14247 entries, 0 to 14246
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   smiles             14247 non-null  object
 1   qsar_ready_smiles  13299 non-null  object
dtypes: object(2)
memory usage: 222.7+ KB


In [52]:
df1 = df[df['screenable']==1]

In [53]:
results = {}
for i, group in df1.groupby('group_str'):
    try:
        a = mk_fp(group)
    except Exception as error:
        print(error)
    else:
        results[i] = a


In [55]:
#results["('Acetylides', nan)"]

In [56]:
distances = {}
for k,v in results.items():
    try:
        a = distance_matrix(results[k])
    except Exception as error:
        print(k, error)
    else:
        distances[k] = a

In [57]:
{k:len(v) for k,v in results.items() if len(v) <= 1}

{"('Homogeneous non-metal compounds', 3.0)": 1,
 "('Lipids and lipid-like molecules', nan)": 1}

In [58]:
cats = {}
for i,group in df1.groupby('group_str'):
    cats[i]  = group[ ['dtxsid'] +df.columns.tolist()[99:]]

In [67]:
cats["('Acetylides', nan)"]

,dtxsid,ClassyFire,cluster_centroid,centroid,centroid2,analogues,Dist,lt65_picks,level2_cluster_centroid,cluster_d,...,MaxMin_selection_picks,category_size,final_picks,visual_inspection_no,Notes,available_round1,sum_avail_classyfire,sum_picks_classyfire,screening_dist_nonzero_low,round2pick
0,DTXSID4063036,Acetylides,centroid,DTXSID4063036,DTXSID4063036,DTXSID4063036,0.000000,1,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,1.0
3,DTXSID9061097,Acetylides,NaN,NaN,DTXSID4063036,DTXSID9061097,0.045455,0,NaN,NaN,...,1.0,between20-70,1.0,NaN,NaN,1.0,2,3,0.045455,NaN
5,DTXSID4061096,Acetylides,NaN,NaN,DTXSID4063036,DTXSID4061096,0.045455,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
6,DTXSID8072405,Acetylides,NaN,NaN,DTXSID4063036,DTXSID8072405,0.419355,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
7,DTXSID0066955,Acetylides,NaN,NaN,DTXSID4063036,DTXSID0066955,0.500000,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
10,DTXSID60870766,Acetylides,NaN,NaN,DTXSID4063036,DTXSID60870766,0.045455,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
11,DTXSID80887937,Acetylides,NaN,NaN,DTXSID4063036,DTXSID80887937,0.617647,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
13,DTXSID30862951,Acetylides,NaN,NaN,DTXSID4063036,DTXSID30862951,0.800000,1,NaN,NaN,...,1.0,between20-70,1.0,NaN,NaN,NaN,2,3,0.045455,NaN
18,DTXSID1022136,Acetylides,NaN,NaN,DTXSID4063036,DTXSID1022136,0.730769,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN
23,DTXSID2060880,Acetylides,NaN,NaN,DTXSID4063036,DTXSID2060880,0.045455,0,NaN,NaN,...,0.0,between20-70,NaN,NaN,NaN,NaN,2,3,0.045455,NaN


In [76]:
def new_picks(df):
    mylst = []
    df = df.set_index('dtxsid')
    for i, row in df.iterrows():
        if ((row['lt65_picks'] ==1) | (row['MaxMin_selection_picks'] ==1)) &  ( np.isnan(row['available_round1'])  ) & (np.isnan(row['Notes'])):
            mylst.append(i)
    return mylst

In [77]:
new_picks(cats["('Acetylides', nan)"])

['DTXSID4063036', 'DTXSID30862951']

In [80]:
mylst2 = []
for k,v in cats.items():
    try:
        a = new_picks(v)
    except Exception as error:
        print(k, error)
    else:
        mylst2.append(a)

('Benzene and substituted derivatives', 3.0) ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
('Diazinanes', nan) ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
('Hydrocarbons', 1.0) ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
('Lactones', nan) ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
('Organic carbonic acids and derivatives', 3.0) ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''
('Organic oxoanionic compounds', nan) ufunc 'isnan' not supported for the inp

In [97]:
#mylst2
#flat_list = [item for sublist in l for item in sublist]

In [90]:
mylst3 = [e for i in mylst2 for e in i]

In [91]:
df['round2_pk_gp'] = df['dtxsid'].apply(lambda x: 1 if x in mylst3 else np.nan)

In [93]:
df.to_csv(interim_dir+'gp_round2.csv')

In [95]:
cats.keys()

dict_keys(["('Acetylides', nan)", "('Alkaloids and derivatives', nan)", "('Anthracenes', 1.0)", "('Anthracenes', 2.0)", "('Anthracenes', 3.0)", "('Anthracenes', 4.0)", "('Anthracenes', 5.0)", "('Anthracenes', 6.0)", "('Anthracenes', 7.0)", "('Anthracenes', 8.0)", "('Anthracenes', 9.0)", "('Azobenzenes', 1.0)", "('Azobenzenes', 2.0)", "('Azobenzenes', 3.0)", "('Azobenzenes', 4.0)", "('Azobenzenes', 5.0)", "('Azobenzenes', 6.0)", "('Azobenzenes', 7.0)", "('Azoles', 1.0)", "('Azoles', 2.0)", "('Azoles', 3.0)", "('Azoles', 4.0)", "('Azoles', 5.0)", "('Azoles', 6.0)", "('Azoles', 7.0)", "('Azolidines', 1.0)", "('Azolidines', 2.0)", "('Azolidines', 3.0)", "('Azolidines', 4.0)", "('Azolidines', 5.0)", "('Azolidines', 6.0)", "('Benzene and substituted derivatives', 1.0)", "('Benzene and substituted derivatives', 10.0)", "('Benzene and substituted derivatives', 2.0)", "('Benzene and substituted derivatives', 3.0)", "('Benzene and substituted derivatives', 4.0)", "('Benzene and substituted deriv

In [96]:
cats["('Alkaloids and derivatives', nan)"][['dtxsid','screenable','centroid', 'Dist', 'lt65_picks', 'MaxMin_selection_picks', 'category_size', 'final_picks',
 'screenable_centroid_id',
 'visual_inspection_no','Notes',
 'screening_dist','available_round1','round2pick'
]]

,dtxsid,screenable,centroid,Dist,lt65_picks,MaxMin_selection_picks,category_size,final_picks,screenable_centroid_id,visual_inspection_no,Notes,screening_dist,available_round1,round2pick
33,DTXSID60883396,1,DTXSID60883396,0.000000,1,0.0,lessthan20,NaN,DTXSID60883396,NaN,NaN,0.000000,NaN,NaN
34,DTXSID0065989,1,NaN,0.927835,1,0.0,lessthan20,NaN,NaN,NaN,NaN,0.927835,NaN,NaN
41,DTXSID30862133,1,NaN,0.912500,1,0.0,lessthan20,NaN,NaN,NaN,NaN,0.912500,NaN,NaN
